# 🧾 Day 8: Train CNN on Custom Dataset (Hinglish)
Aaj hum custom image dataset ka use karenge CNN ko train karne ke liye.

## 📁 Dataset Structure (Example)
```
custom_data/
├── train/
│   ├── class1/
│   └── class2/
├── test/
    ├── class1/
    └── class2/
```
> Har class ke images apne alag folder mein hone chahiye.

In [ ]:
# ✅ Import karo
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

## 🔃 Load Data using ImageDataGenerator
- Images ko normalize karte hain (1/255)
- `target_size=(64, 64)` ka matlab har image ko 64x64 size mein convert karna

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_data = train_datagen.flow_from_directory(
    'custom_data/train',
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical'
)

test_data = test_datagen.flow_from_directory(
    'custom_data/test',
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical'
)

## 🧠 CNN Model for Custom Dataset
- Input shape = (64, 64, 3) → RGB image
- Output neurons = number of classes (dynamic from dataset)

In [ ]:
num_classes = train_data.num_classes

model = keras.Sequential([
    keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(64,64,3)),
    keras.layers.MaxPooling2D(2,2),
    keras.layers.Conv2D(64, (3,3), activation='relu'),
    keras.layers.MaxPooling2D(2,2),
    keras.layers.Flatten(),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(num_classes, activation='softmax')
])

## 🏋️ Model Train karo
- Categorical Crossentropy because of multiple classes
- Adam optimizer + accuracy metric
- 10 epochs
- Model save karo 'custom_model.h5'

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_data, epochs=10, validation_data=test_data)
model.save('custom_model.h5')
#✅ Model ko save karo
print("Model saved as 'custom_model.h5'")  # ✅ Confirmation message

## Run and Test the model

In [ ]:
# ✅ Import required libraries
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# ✅ Load the saved model
model = keras.models.load_model('custom_model.h5')
print("✅ Model loaded successfully!")

# ✅ Prepare test data
test_datagen = ImageDataGenerator(rescale=1./255)
test_data = test_datagen.flow_from_directory(
    'custom_data/test',
    target_size=(64, 64),
    batch_size=1,  # Predict 1 image at a time
    class_mode='categorical',
    shuffle=False
)

# ✅ Get class labels (cat & dog)
class_indices = test_data.class_indices
labels = list(class_indices.keys())  # ['cat', 'dog']

# ✅ Predict on test data
predictions = model.predict(test_data, verbose=1)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = test_data.classes

# ✅ Test accuracy
accuracy = np.sum(predicted_classes == true_classes) / len(true_classes)
print(f"✅ Test Accuracy: {accuracy * 100:.2f}%")

# ✅ Visualize Predictions
def visualize_predictions(test_data, predicted_classes, labels, num_images=None):
    if num_images is None or num_images > len(predicted_classes):
        num_images = len(predicted_classes)

    # Reset generator to start from the beginning
    test_data.reset()
    
    plt.figure(figsize=(15, 8))
    for i in range(num_images):
        img_batch, true_label_batch = next(test_data)
        img = img_batch[0]
        true_label = np.argmax(true_label_batch[0])
        pred_label = predicted_classes[i]
        
        plt.subplot(2, 5, i+1)
        plt.imshow(img)
        title_color = 'green' if pred_label == true_label else 'red'
        plt.title(f"True: {labels[true_label]}\nPred: {labels[pred_label]}", color=title_color)
        plt.axis('off')

    plt.tight_layout()
    plt.show()
# ✅ Visualize predictions
visualize_predictions(test_data, predicted_classes, labels)
